#### Monte Carlo Simulation

In [1]:
# Initial imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np
import hvplot.pandas
import plotly.express as px


%matplotlib inline

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# Load .env enviroment variables
load_dotenv()

In [ ]:
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_SECRET_KEY")

In [ ]:
# Set start and end dates of five years back from today.
# Formatting current date as ISO format
start_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-02-14", tz="America/New_York").isoformat()

In [ ]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [ ]:

# Setting the tickers
tickers = ["FB","AMZN","NFLX","GOOG","SPY"]

# Setting timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Getting current closing prices for the tickers
df_tech_stock_data = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df


# Reorganize the DataFrame
# Separate ticker data
FB = df_tech_stock_data[df_tech_stock_data['symbol']=='FB'].drop('symbol', axis=1)
# APPL = df_tech_stock_data[df_tech_stock_data['symbol']=='APPL'].drop('symbol', axis=1)
SPY = df_tech_stock_data[df_tech_stock_data['symbol']=='SPY'].drop('symbol', axis=1)
AMZN = df_tech_stock_data[df_tech_stock_data['symbol']=='AMZN'].drop('symbol', axis=1)
NFLX = df_tech_stock_data[df_tech_stock_data['symbol']=='NFLX'].drop('symbol', axis=1)
GOOG = df_tech_stock_data[df_tech_stock_data['symbol']=='GOOG'].drop('symbol', axis=1)


# Concatenate the ticker DataFrames
df_tech_stock_data = pd.concat([GOOG,SPY,AMZN,NFLX,FB],axis=1, keys=['GOOG','SPY','AMZN','NFLX','FB'])

# Preview DataFrame
df_tech_stock_data.head()
df_tech_stock_data.tail()


In [ ]:
df_tech_stock_data.dropna()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
weights = np.array([.2,.2,.2,.2,.2])

thirty_years_cumulative_returns = MCSimulation(
    portfolio_data = df_tech_stock_data ,
    weights=[.2,.2,.2,.2,.2],
    num_simulation=500,
    num_trading_days=252*30
)
thirty_years_cumulative_returns

In [ ]:
thirty_years_cumulative_returns.portfolio_data

In [ ]:
thirty_years_cumulative_returns.calc_cumulative_return()

In [ ]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-02-14", tz="America/New_York").isoformat()
# Set the tickers
tickers = ["VOO", "VGT",'SPY']

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for VOO and VGT
df_etf = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
VOO = df_etf[df_etf['symbol']=='VOO'].drop('symbol', axis=1)
VGT = df_etf[df_etf['symbol']=='VGT'].drop('symbol', axis=1)
SPY = df_etf[df_etf['symbol']=='SPY'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
df_etf = pd.concat([VOO,VGT,SPY],axis=1, keys=['VOO','VGT','SPY'])

# Preview DataFrame
df_etf.head()
df_etf.tail()

In [ ]:
df_etf.dropna()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
thirty_years_cumulative_returns = MCSimulation(
    portfolio_data = df_etf ,
    weights=[.3,.3,.4],
    num_simulation=500,
    num_trading_days=252*30
)
thirty_years_cumulative_returns

In [ ]:
thirty_years_cumulative_returns.portfolio_data

In [ ]:
thirty_years_cumulative_returns.calc_cumulative_return()